In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [27]:
!nvidia-smi


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Tue Oct 22 20:48:25 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P0             32W /  250W |    1149MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [28]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import time
from joblib import Parallel, delayed
import multiprocessing


In [29]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Set the model to use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Check if using GPU
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Using CPU")


Using GPU: Tesla P100-PCIE-16GB


In [30]:
url = 'https://raw.githubusercontent.com/amanullahshah32/Review-Scraping/refs/heads/main/Dataset/all_app_reviews.csv'
df = pd.read_csv(url)


In [31]:
df_shuffled = df.sample(frac=1).reset_index(drop=True)
df_sampled = df_shuffled.sample(n=10000, random_state=42)


In [32]:
df_review_description = df_sampled[['review_description']]


In [33]:
full_df = df[['review_description']]

In [34]:
full_df

,review_description
0,nice
1,good
2,good
3,good
4,good 😊
...,...
99390,wow
99391,র্গববতী গরু খিচনি রোগ চিকিছ্যা পরামর্শ
99392,good
99393,"ফালতু অ্যাপ, ওষুধের ডোজ কতটুকু তা দেওয়া নাই"


In [35]:
len(full_df)

99395

In [36]:
df_review_description.head()

,review_description
43883,nice to
21124,Excellent
29732,"Coming from someone studying medicine, in my o..."
88813,Really good
28576,Very easy to use.


In [37]:
def sentiment_score(review):
    # Handle missing or non-string values by converting to empty strings
    if not isinstance(review, str):
        review = ''
    
    # Tokenize and move the tokens to GPU
    tokens = tokenizer.encode(review, return_tensors='pt').to(device)
    
    # Perform inference on GPU
    result = model(tokens)
    
    # Return the sentiment score (1-5 scale)
    return int(torch.argmax(result.logits)) + 1


In [38]:
# Step 1: Clean the DataFrame by replacing non-string values
df_review_description['review_description'] = df_review_description['review_description'].apply(lambda x: x if isinstance(x, str) else '')

# Step 2: Measure the start time
start_time = time.time()

# Step 3: Apply sentiment scoring
df_review_description['sentiment'] = df_review_description['review_description'].apply(lambda review: sentiment_score(review[:512]))

# Step 4: Measure the end time and print the total time taken
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")


/tmp/ipykernel_30/101572710.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review_description['review_description'] = df_review_description['review_description'].apply(lambda x: x if isinstance(x, str) else '')


Time taken: 95.83065676689148 seconds


/tmp/ipykernel_30/101572710.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review_description['sentiment'] = df_review_description['review_description'].apply(lambda review: sentiment_score(review[:512]))


In [39]:
df_review_description

,review_description,sentiment
43883,nice to,4
21124,Excellent,5
29732,"Coming from someone studying medicine, in my o...",3
88813,Really good,5
28576,Very easy to use.,5
...,...,...
44546,RMG doctor from Khulshi koloni purbo naserabad,1
72889,টিকা নিবন্ধনের সময় যে ফোন নাম্বার দিয়ে নিবন্ধন...,2
8781,I can find all the information I need on this ...,5
36752,awesome app,5


In [45]:
df_review_description.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 43883 to 84245
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   review_description  10000 non-null  object
 1   sentiment           10000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 492.4+ KB


## Sentiment on full dataset:


In [40]:
# Step 1: Clean the DataFrame by replacing non-string values
full_df['review_description'] = full_df['review_description'].apply(lambda x: x if isinstance(x, str) else '')

# Step 2: Measure the start time
start_time = time.time()

# Step 3: Apply sentiment scoring
full_df['sentiment_full_df'] = df_review_description['review_description'].apply(lambda review: sentiment_score(review[:512]))

# Step 4: Measure the end time and print the total time taken
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")


/tmp/ipykernel_30/2256262366.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df['review_description'] = full_df['review_description'].apply(lambda x: x if isinstance(x, str) else '')


Time taken: 98.09001207351685 seconds


/tmp/ipykernel_30/2256262366.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df['sentiment_full_df'] = df_review_description['review_description'].apply(lambda review: sentiment_score(review[:512]))


In [41]:
full_df.sentiment_full_df.value_counts()

sentiment_full_df
5.0    5683
4.0    2551
1.0     829
3.0     690
2.0     247
Name: count, dtype: int64

In [42]:
full_df.sentiment_full_df.isnull()

0        True
1        True
2        True
3        True
4        True
         ... 
99390    True
99391    True
99392    True
99393    True
99394    True
Name: sentiment_full_df, Length: 99395, dtype: bool

In [43]:
# Step 5: Remove rows where sentiment_full_df is null
full_df_cleaned = full_df.dropna(subset=['sentiment_full_df'])

# Verify that no null values are present
print(f"Number of nulls in 'sentiment_full_df': {full_df_cleaned['sentiment_full_df'].isnull().sum()}")

# Optional: Show the shape of the cleaned DataFrame to see how many rows are left
print(f"Cleaned DataFrame shape: {full_df_cleaned.shape}")

Number of nulls in 'sentiment_full_df': 0
Cleaned DataFrame shape: (10000, 2)


In [44]:
full_df_cleaned

,review_description,sentiment_full_df
23,There is no full version of the system l think...,5.0
53,very good,5.0
70,good,5.0
87,Wonderful useful app,5.0
97,Good,4.0
...,...,...
99334,good,5.0
99349,Ato add dekhai...uninstall kore dilam,5.0
99355,really so important,4.0
99381,good,5.0
